Example from [Neuroglancer demo](https://github.com/google/neuroglancer/blob/master/python/examples/jupyter-notebook-demo.ipynb) implemented below with custom made ipywidgets (Cell 8).

This is a [Neuroglancer documentation](https://connectomics.readthedocs.io/en/latest/external/neuroglancer.html) to help with Neuroglancer installation. 


In [1]:
import neuroglancer
import numpy as np

This cell starts a webserver in a background thread, which serves a copy of the Neuroglancer client, and which also can serve local volume data and handles sending and receiving Neuroglancer state updates.

In [2]:
viewer = neuroglancer.Viewer()

This cell adds 2 example layers. 
This particular cell uses the precomputed data source (HHMI Janelia FlyEM FIB-25 dataset), but the source can be from anywhere. 
The `.txn()` method performs a state-modification transaction (changes the view state).

In [3]:
with viewer.txn() as s:
  s.layers['image'] = neuroglancer.ImageLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/image')
  s.layers['segmentation'] = neuroglancer.SegmentationLayer(source='precomputed://gs://neuroglancer-public-data/flyem_fib-25/ground_truth', selected_alpha=0.3)
  

Move the viewer position.

In [4]:
with viewer.txn() as s:
    s.voxel_coordinates = [3000.5, 3000.5, 3000.5]

Select a couple segments.

In [5]:
with viewer.txn() as s:
    s.layers['segmentation'].segments.update([1752, 88847])

Update the state by calling `set_state` directly and provide new state uid. `set_state` overrides entire state and returns somethng like state uid.

In [6]:
import copy
new_state = copy.deepcopy(viewer.state)
new_state.layers['segmentation'].segments.add(10625)
viewer.set_state(new_state)

'e9f43963e26c0988fd960f79ad9dca5de2093fe3'

Change the view layout to 3-d and set projection scale.

Be aware, it has something like a bug when we try to set projection scale before we call `display()` (cell 8), it doesn't apply the value the first time. However, if manually run it after running `display()`, it will set the projection scale correctly. 

In [10]:
with viewer.txn() as s:
    s.layout = '3d'
    s.projection_scale = 3000

The cell below creates custom widgets in order to show the Navigation Panel on the left of the viewer to manipulate the data. 

In [8]:
import ipywidgets as widgets

def get_header(text):
    return widgets.HTML("<h3>{}</h3>".format(text), layout=widgets.Layout(height='auto'))

# Widgets

# Zoom
zoom_slider = widgets.FloatSlider(
    value=1,
    min=1,
    max=25,
    step=1,
    description='Zoom:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# X-Axis
x_location = widgets.FloatSlider(
    value=3000,
    min=1500,
    max=4500,
    step=100,
    description='X Location:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Y-Axis

y_location = widgets.FloatSlider(
    value=3000,
    min=1500,
    max=4500,
    step=100,
    description='Y Location:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

#Z-Axis

z_location = widgets.FloatSlider(
    value=1000,
    min=1000,
    max=10000,
    step=100,
    description='Z Location:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)

# Layers

layers = [
    widgets.Checkbox(
        value=True,
        description='Image',
        disabled=False,
        indent=False
    ),
    widgets.Checkbox(
        value=True,
        description='Segmentation',
        disabled=False,
        indent=False
    ),
]

# add widget to change layout
current_layout=widgets.ToggleButtons(
    options=["xy","yz","xz","4panel","3d"],                           
    disabled=False,
    value="3d",
    # layout=widgets.Layout(width='50%')
)

# Callback list

def change_zoom_value(change):
    with viewer.txn() as s:
        s.crossSectionScale = change['new']

def change_x_axis(change):
    with viewer.txn() as s:
        s.position[0] = change['new']
        
def change_y_axis(change):
    with viewer.txn() as s:
        s.position[1] = change['new']
        
def change_z_axis(change):
    with viewer.txn() as s:
        s.position[2] = change['new']
        
def change_layer(change):
    layer_name = change.owner.description.lower()
    with viewer.txn() as s:
        layer = [l for l in s.layers if l.name == layer_name]
        if len(layer) == 1:
            layer[0].visible = change['new']
            
def layout_observer(change):
    with viewer.txn() as s:
        s.layout = change['new']
        
# Observers
    
zoom_slider.observe(change_zoom_value, 'value')
x_location.observe(change_x_axis, 'value')
y_location.observe(change_y_axis, 'value')
z_location.observe(change_z_axis, 'value')

for child in layers:
    child.observe(change_layer, 'value')
    
current_layout.observe(layout_observer,'value')
    
# Display

htmlIframe = '<iframe src="{}" style="width:99%;height:98vh;"></iframe>'.format(str(viewer))

iframe_widget = widgets.HTML(value=htmlIframe,layout=widgets.Layout(width='100%',height='100%'))
display(widgets.HBox(
    children=[widgets.VBox([
        get_header("Navigation"),
        zoom_slider,
        x_location,
        y_location,
        z_location,
        get_header("Layers"),
        *layers,
        get_header("Layout"),
        current_layout
    ],
    layout=widgets.Layout(width='430px',height='100%')
), iframe_widget]))


Stop the Neuroglancer web server, which invalidates any existing links to the Python-tied viewer, run the command in the cell below. 

In [9]:
neuroglancer.stop()